# Data Analysis
## Tweet data

In [2]:
import sqlite3
import pandas as pd
from afinn import Afinn
import json

import sys
sys.path.insert(0, "../src")
import tjson

# Fetch data from db
db = sqlite3.connect('../tweets.db')
c = db.cursor()

In [16]:
c.execute("SELECT * FROM tweets")

rows = c.fetchall()

tweets = pd.DataFrame(columns=['id', 'date', 'json', 'filter'], data=rows)
tweets['date'] = pd.to_datetime(tweets['date'],format='%Y-%m-%d %H:%M:%S')

In [17]:
afinn = Afinn()
tweets['sentiment_score'] = tweets['json'].apply(lambda x: afinn.score(tjson.tweet_text(x)))
tweets['text'] = tweets['json'].apply(lambda x: tjson.tweet_text(x))
tweets['retweet'] = tweets['json'].apply(lambda x: tjson.is_retweet(x))
tweets['symbols'] = tweets['json'].apply(lambda x: tjson.tweet_symbols(x))

In [18]:
pd.options.display.max_colwidth = 3000
tweets[['sentiment_score','text','retweet']][tweets['sentiment_score']>0].sample(5)


,sentiment_score,text,retweet
621,4.0,RT @futureearn: Check this amazing game and enter in following #giveaway\nhttps://t.co/fUSgAGvcfj …\n2 $NEO giveaway to random person retweet…,True
529,2.0,"RT @Aruwba: 💥FLASH SALE!💥\n Come join my uncensored snap, with fresh daily posts! \n🚨Get it now for ONLY $25 FOR LIFE🚨\n\n⬇️Click here⬇️ \nhttps…",True
560,4.0,RT @bitcommander: Best currency exchange rate converter for iPhone. #free #ios #fx #currency $USD $EUR $GBP $BTC and more. @HyperRTs @Night…,True
189,1.0,"RT @Blockchainlife: 2010: Laszlo Hanyecz agreed to give a guy in the UK 10,000 $BTC for ordering him a pizza from Papa Johns and had it de…",True
456,6.0,"RT @GloryCrypto: RT &amp;I follow for a chance to win 2 spots in my SPECIAL Premium paidChat , I will be announcing 2 winners for the last spot…",True


In [19]:
tweets[['retweet','text','symbols','filter']].sample(5)

,retweet,text,symbols,filter
288,False,@money2020 can payment be made with #bitcoin? $btc #Money2020 #cryptocurrency @monaco_card,"[{'text': 'btc', 'indices': [46, 50]}]",$btc
233,False,#Bitcoin Update: ABC structure before Wave 3? $BTC #crypto https://t.co/96xNq83mYC,"[{'text': 'BTC', 'indices': [46, 50]}]",$btc
611,False,"Working on a solid game plan is okay. \n\nBut, sticking to the game plan, i.e. commitment will determine the winners from the losers, almost every time.\n\n#Cryptocurrency #Blockchain\n\n$BTC $ETH $BCH $MIOTA $LTC $XRP $DASH $BTG $XMR $XEM $ADA $XLM $EOS $BTG $NEO $QTUM",[],$eth
92,True,"RT @GloryCrypto: RT &amp;I follow for a chance to win 2 spots in my SPECIAL Premium paidChat , I will be announcing 2 winners for the last spot…",[],$btc
500,False,Want to know what coin to hold?\n\nhttps://t.co/6jTc5MopCs\n\n$VOX $NMR $DCR $LBC $ARK $EFL $BTC $DGB $NXS $DASH $WINGS $NAV $BLOCK $EDG $ZEC $SHIFT $PTOY $BCH $STORJ $PIVX $ETH $STEEM $NXT $XZC $CVC $ADA $NEO $MONA $KMD $ENRG \nnZ2DAPNBUJ https://t.co/3phWr75J7L,"[{'text': 'VOX', 'indices': [58, 62]}, {'text': 'NMR', 'indices': [63, 67]}, {'text': 'DCR', 'indices': [68, 72]}, {'text': 'LBC', 'indices': [73, 77]}, {'text': 'ARK', 'indices': [78, 82]}, {'text': 'EFL', 'indices': [83, 87]}, {'text': 'BTC', 'indices': [88, 92]}, {'text': 'DGB', 'indices': [93, 97]}, {'text': 'NXS', 'indices': [98, 102]}, {'text': 'DASH', 'indices': [103, 108]}, {'text': 'WINGS', 'indices': [109, 115]}]",$btc


## Processed Data with different time resolution

In [20]:
c.execute("SELECT * FROM vart_10min")

rows = c.fetchall()
varT= pd.DataFrame(columns=['date', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13'], data=rows)

In [26]:
varT[varT.f1 != 0]

,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13
0,2018-02-27 18:30:00,13,0,0,6,4,0,0,5,8,2,3,5,0
85,2018-02-28 08:40:00,84,10,3,51,44,10,2,31,23,8,53,14,38
86,2018-02-28 08:50:00,128,12,8,57,50,11,1,59,38,16,74,24,41
87,2018-02-28 09:00:00,131,12,5,58,42,4,1,61,41,13,77,23,39
88,2018-02-28 09:10:00,79,9,9,39,29,8,4,38,32,7,40,16,23
273,2018-03-01 16:00:00,5,2,0,2,2,2,0,3,5,0,0,2,0
274,2018-03-01 16:10:00,26,3,1,11,6,2,1,16,14,4,8,9,4
275,2018-03-01 16:20:00,21,1,1,4,2,0,0,17,11,8,2,6,1
276,2018-03-01 16:30:00,36,6,3,18,15,6,0,22,16,3,17,10,10
